In [1]:
import pickle
import pandas as pd
import numpy as np
import time

import dataloader
import evaluate
import model
import training
import influence

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna

from importlib import reload
device='cpu'

In [2]:
data_dir = './data'
dataset = dataloader.AmazonDataset(data_dir)
evaluater = evaluate.Evaluater(data_dir)


# 学習済みモデル読み込み
mf = torch.load('model.torch')
# loss_func定義
loss_func = nn.BCELoss()

# 任意のテストデータ[u_i, i_i]ロード
# あるユーザに対してランキング上位のアイテム
target_user = dataset.user_list[0]
target_user = dataset.user_list.index(target_user)
ranking_idx = evaluater.predict(mf, target_user)
target_item = ranking_idx[0]


# trainingデータのinfluenceを計算
## [u_i, i_i]のどちらかを含むtrainingデータを持ってくる
train_data = []
for row in dataset.user_item_train_df.values:
    if target_user == row[0] or target_item == row[1]:
        train_data.append(row)


In [20]:
reload(influence)
test_data = [target_user, target_item]
influ = influence.get_influence(loss_func, train_data[0], test_data, mf)
influ

tensor(-2.1051e-07, grad_fn=<DivBackward0>)

In [40]:
len(dataset.user_list) * 65 + len(dataset.item_list) * 65

351000

In [146]:
def get_loss(loss_func, train_data, model):
    user = torch.tensor([train_data[0]], dtype=torch.long, device=device)
    item = torch.tensor([train_data[1]], dtype=torch.long, device=device)
    pred = model(user, item)
    y = torch.tensor([1.])
    loss = loss_func(pred, y)

    return loss

def get_loss_gradient(model, loss):
    grads = torch.autograd.grad(loss, model.parameters(), create_graph = True, retain_graph = True)
    grad = []
    for g in grads:
        tmp = g.view(1, g.shape[0] * g.shape[1])
        grad.append(tmp)

    return torch.cat(grad, dim=1)[0]

def hessian_vector_product_2(model, grad_loss, v):
    w = torch.dot(grad_loss, v)
    w.backward()
    params = model.parameters()
    hv = []
    for p in params:
        hv.append(p.grad.view(1, p.shape[0]*p.shape[1]))

    return torch.cat(hv, dim=1)[0]

def hessian_vector_product(model, grad_loss, v):
    w = torch.dot(grad_loss, v)
    grads = torch.autograd.grad(w, model.parameters(), retain_graph=True)
    grad = []
    for g in grads:
        tmp = g.view(1, g.shape[0] * g.shape[1])
        grad.append(tmp)

    return torch.cat(grad, dim=1)[0]

In [161]:
def get_influence(loss_func, train_data, test_data, model):
    loss = get_loss(loss_func, train_data, model)

    grad_loss = get_loss_gradient(model, loss)
    print(grad_loss)
    v = torch.ones(grad_loss.size())
    hv = hessian_vector_product(model, grad_loss, v)
    print(hv.shape)
    #hv2 = hessian_vector_product_2(model, grad_loss, v)

    hv3 = hessian_vector_product(model, grad_loss, v)
    #print(torch.norm(hv - hv2))
    print(torch.norm(hv - hv3))
    #return h_inverse_v

In [162]:
mf = torch.load('model.torch')
get_influence(loss_func, train_data[0], test_data, mf)

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SelectBackward>)
torch.Size([351000])
tensor(0.)
